In [21]:
import pandas as pd

# Load datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

# Inspect datasets
print(customers.head())
print(products.head())
print(transactions.head())

# Check for null values and duplicates
print(customers.isnull().sum(), products.isnull().sum(), transactions.isnull().sum())
print(customers.duplicated().sum(), products.duplicated().sum(), transactions.duplicated().sum())


  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3       

In [8]:
# Merge datasets
merged = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

# Inspect the merged data
print(merged.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving Bluetooth Speaker

In [10]:
# Customer-level aggregates
customer_features = merged.groupby('CustomerID').agg({
    'TotalValue': ['sum', 'mean'],        # Total and average spending
    'TransactionID': 'count',            # Transaction count
    'ProductID': pd.Series.nunique,      # Unique products
    'Category': lambda x: x.mode()[0]    # Most common category
}).reset_index()

# Rename columns
customer_features.columns = [
    'CustomerID', 'TotalSpending', 'AvgTransactionValue',
    'TransactionCount', 'UniqueProducts', 'CommonCategory'
]

# One-hot encode categorical data (e.g., CommonCategory)
customer_features = pd.get_dummies(customer_features, columns=['CommonCategory'], drop_first=True)

print(customer_features.head())


  CustomerID  TotalSpending  AvgTransactionValue  TransactionCount  \
0      C0001        3354.52              670.904                 5   
1      C0002        1862.74              465.685                 4   
2      C0003        2725.38              681.345                 4   
3      C0004        5354.88              669.360                 8   
4      C0005        2034.24              678.080                 3   

   UniqueProducts  CommonCategory_Clothing  CommonCategory_Electronics  \
0               5                    False                        True   
1               4                     True                       False   
2               4                    False                       False   
3               8                    False                       False   
4               3                    False                        True   

   CommonCategory_Home Decor  
0                      False  
1                      False  
2                       True  
3         

In [12]:
from sklearn.preprocessing import StandardScaler

# Define numerical columns
numerical_columns = ['TotalSpending', 'AvgTransactionValue', 'TransactionCount', 'UniqueProducts']

# Normalize features
scaler = StandardScaler()
customer_features[numerical_columns] = scaler.fit_transform(customer_features[numerical_columns])

print(customer_features.head())


  CustomerID  TotalSpending  AvgTransactionValue  TransactionCount  \
0      C0001      -0.061701            -0.070263         -0.011458   
1      C0002      -0.877744            -0.934933         -0.467494   
2      C0003      -0.405857            -0.026271         -0.467494   
3      C0004       1.032547            -0.076769          1.356650   
4      C0005      -0.783929            -0.040028         -0.923530   

   UniqueProducts  CommonCategory_Clothing  CommonCategory_Electronics  \
0        0.050047                    False                        True   
1       -0.424204                     True                       False   
2       -0.424204                    False                       False   
3        1.472798                    False                       False   
4       -0.898455                    False                        True   

   CommonCategory_Home Decor  
0                      False  
1                      False  
2                       True  
3         

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Drop CustomerID for similarity calculation
feature_matrix = customer_features.drop(columns=['CustomerID']).values

# Calculate cosine similarity
similarity_matrix = cosine_similarity(feature_matrix)

# Convert similarity matrix into a DataFrame for readability
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=customer_features['CustomerID'],
    columns=customer_features['CustomerID']
)

print(similarity_df.head())


CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000  0.059268  0.008758 -0.000067  0.558583 -0.086717   
C0002       0.059268  1.000000  0.357228 -0.532381  0.487216 -0.470988   
C0003       0.008758  0.357228  1.000000 -0.594532  0.500131  0.087284   
C0004      -0.000067 -0.532381 -0.594532  1.000000 -0.828985 -0.243356   
C0005       0.558583  0.487216  0.500131 -0.828985  1.000000  0.135587   

CustomerID     C0007     C0008     C0009     C0010  ...     C0191     C0192  \
CustomerID                                          ...                       
C0001       0.507631  0.039413  0.061549  0.063597  ...  0.097551  0.681421   
C0002       0.173518 -0.226706  0.970074  0.997878  ...  0.702368  0.630461   
C0003       0.428229 -0.270247  0.421638  0.348681  ...  0.172782  0.368348   
C0004      -0.751216  0.870160 -0.639825 -0.513210  ... -0.174444 -0.559980   
C0005  

In [16]:
# Generate lookalike recommendations
lookalike_results = {}
for customer_id in customer_features['CustomerID'][:20]:  # First 20 customers
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:4]
    lookalike_results[customer_id] = list(zip(similar_customers.index, similar_customers.values))

# Print recommendations
print(lookalike_results)


{'C0001': [('C0072', 0.9462192572233441), ('C0190', 0.9416801751498163), ('C0069', 0.9107245938690862)], 'C0002': [('C0029', 0.9993654241194002), ('C0010', 0.9978779681739283), ('C0009', 0.970074440627757)], 'C0003': [('C0178', 0.9998807764023382), ('C0166', 0.9617406117799829), ('C0052', 0.9438158596050766)], 'C0004': [('C0021', 0.9997854801171627), ('C0075', 0.9995851081107424), ('C0175', 0.9977423749442703)], 'C0005': [('C0112', 0.9981708762371513), ('C0197', 0.9963907311258234), ('C0095', 0.9836446116534969)], 'C0006': [('C0117', 0.9978898632254388), ('C0168', 0.9880664242438316), ('C0185', 0.9472256882616829)], 'C0007': [('C0120', 0.9961287863517073), ('C0140', 0.9836919191804516), ('C0020', 0.950021083285971)], 'C0008': [('C0194', 0.9585048932099444), ('C0090', 0.9533667364360966), ('C0139', 0.9526998939977023)], 'C0009': [('C0077', 0.9997756842715478), ('C0083', 0.9964828112163444), ('C0010', 0.9763477987228947)], 'C0010': [('C0029', 0.9995639927786868), ('C0002', 0.997877968173

In [18]:
# Prepare data for CSV
lookalike_data = []
for customer, lookalikes in lookalike_results.items():
    row = [customer]
    for lookalike_id, score in lookalikes:
        row.extend([lookalike_id, score])
    lookalike_data.append(row)

# Create DataFrame
lookalike_df = pd.DataFrame(
    lookalike_data,
    columns=['CustomerID', 'LookalikeID1', 'Score1', 'LookalikeID2', 'Score2', 'LookalikeID3', 'Score3']
)

# Save to CSV
lookalike_df.to_csv('Lookalike.csv', index=False)
